In [4]:
import numpy as np
import matplotlib as plt

In [5]:
# DAG provided in appendix A
dag = {
    (0, 30): 1,
    (1, 0): 1,
    (2, 7): 1,
    (3, 2): 1,
    (4, 1): 1,
    (5, 15): 1,
    (6, 5): 1,
    (7, 6): 1,
    (8, 7): 1,
    (9, 8): 1,
    (10, 0): 1,
    (11, 4): 1,
    (12, 11): 1,
    (13, 12): 1,
    (16, 14): 1,
    (14, 10): 1,
    (15, 4): 1,
    (16, 15): 1,
    (17, 16): 1,
    (18, 17): 1,
    (19, 18): 1,
    (20, 17): 1,
    (21, 20): 1,
    (22, 21): 1,
    (23, 4): 1,
    (24, 23): 1,
    (25, 24): 1,
    (26, 25): 1,
    (27, 25): 1,
    (28, 26): 1,
    (28, 27): 1,
    (29, 3): 1,
    (29, 9): 1,
    (29, 13): 1,
    (29, 19): 1,
    (29, 22): 1,
    (29, 28): 1
}

# processing times provided in Appendix A
p = [3, 10, 2, 2, 5, 2, 14, 5, 6, 5, 5, 2, 3, 3, 5, 6, 6, 6, 2, 3, 2, 3, 14, 5, 18, 10, 2, 3, 6, 2, 10]

# due dates provided in Appendix A
d = [172, 82, 18, 61, 93, 71, 217, 295, 290, 287, 253, 307, 279, 73, 355, 34,
     233, 77, 88, 122, 71, 181, 340, 141, 209, 217, 256, 144, 307, 329, 269]

In [ ]:
def lcl_rule(dag, processing_times):
    all_schedules = []
    schedule = []
    return schedule, all_schedules

In [ ]:
def generate_neighboring_schedules(dag, schedule):
    neighboring_schedule = []
    return neighboring_schedule

def tabu_search(dag, processing_times, due_dates, num_iterations=100, tolerance=10, tabu_list_length=20):
    tabu_list = np.zeros(tabu_list_length)
    schedule = []
    total_tardiness = 0
    return schedule, total_tardiness

In [ ]:
tolerances = [0, 5, 10, 50, 100, 200, 500]
tabu_list_lengths = [5, 10, 20, 50, 100]
num_iters = [10, 100, 1000, 5000]

for tolerance in tolerances:
    _, total_tardiness = tabu_search(dag, p, d, tolerance=tolerance)
    # TODO: plot total tardiness vs tolerance
    
for tabu_len in tabu_list_lengths:
    _, total_tardiness = tabu_search(dag, p, d, tabu_list_length=tabu_len)
    # TODO: plot total tardiness vs tabu list length
    
for num_iter in num_iters:
    _, total_tardiness = tabu_search(dag, p, d, num_iter=num_iter)
    # TODO: plot total tardiness vs num iterations
    